# Template de entrega

El propósito de este template es que la organización del evento pueda ejecutar el/los prompts generados por el grupo sobre el conjunto de preguntas de evaluación que irían en el mismo formato que las que aparecen en este ejemplo.

Este documento es sólo una plantilla. Cada grupo puede modificarla de la manera que considere. El único requisito es que el notebook entregado sea capaz de procesar un fichero **"preguntas.json"** que tenga el mismo formato que el que aparece en este repositorio. Ya que la evaluación de los prompts se realizará con un fichero **"preguntas_evaluación.json"** confidencial que tiene el mismo formato.

In [1]:
import json
import copy
from pathlib import Path

El fichero **"prompts.json"** debe incluir uno o varios prompts en caso de que el grupo haya decidido responder al desafío con un único prompt o haya decidido generar un prompt específico para cada módulo del examen de acceso.

En el fichero **"prompts.json"** proporcionado en este repositorio hemos incluido 5 prompts, uno para cada categoría del examen de acceso. Notar que los 4 primeros prompts son el mismo y el quinto prompt es algo más elaborado. 

In [2]:
# Read the JSON files
with open("prompts/prompts.json", 'r', encoding='utf-8') as file:
    prompts = json.load(file)

with open("preguntas/preguntas.json", 'r') as file:
    preguntas = json.load(file)['preguntas']

La función **"ejecutar_prompt"** debe implemenetar las llamadas correpondientes al API del LLM que se esté utilizando. En caso de que el LLM no disponga de un API, se debe contactar con lfsanchez@comillas.edu para averiguar la mejor forma de automatizar la toma de resultados del conjunto de preguntas de evaluación

In [3]:
def ejecutar_prompt(prompt, pregunta):
    # Esta función debería ser modificada para realizar llamadas reales a APIs de modelos de lenguaje.
    # La siguiente es una simulación de respuesta.
    respuesta_simulada = "Esto es una respuesta simulada."
    justificacion_simulada = "Esta es una justificación simulada."
    return respuesta_simulada, justificacion_simulada

En el caso del API de OpenAI para chatGPT 3.5 el código se muestra a continuación, aunque el primer paso es instalar la librería que implementa las funciones de llamada a su API:

In [4]:
# !pip install openai

Una vez que disponemos de la librería y de un API key generado desde un perfil de OpenAI, un código como el que se presenta a continuación permitiría utilizar ChatGPT 3.5 de manera programática.

La implementación de la función **ejecutar_prompt** no es única. Lo único importante es que la función reciba el prompt y la consulta y la función devuelva el resultado y la justificación.

In [5]:
import os
from openai import OpenAI

def ejecutar_prompt(prompt, question):
    # Descomentar la línea del api_key y utilizar la clave real
    client = OpenAI(
        #api_key="API KEY FOR OPEN AI",
    )
    
    # Reemplaza {} por la pregunta que debe ir en el prompt
    for message in prompt:
        if message['role'] == 'user' and "{}" in message['content']:
            message['content'] = message['content'].replace("{}", question)

    # Invoca al modelo de lenguaje. Notar que se mete en un bucle infinito porque para poder interpreta los resultados
    # la consulta debe devolver la salida en el formato que le pedimos. Si el formato no es correcto,
    # volvemos a repetir la consulta
    while True:
        response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=prompt,
          max_tokens=150,
          temperature=0.7
        )
        try:
            respuesta, justificacion = response.choices[0].message.content.split("^")
            print("  Done")
            break
        except Exception as e: 
            print(e)

    return respuesta, justificacion

Por último, esta porción de código recorre todas las preguntas del fichero **"preguntas.txt"** y todos los prompts del fichero **"prompts.json"** e invoca a **"ejecutar_prompt"** para cada combinación. El código recoje la respuesta del LLM en forma de una tupla (respuesta, justificación) que se utiliza para evaluar el resultado de la entrega.

Dependiendo de si el fichero **"prompts.json"** incluye un único prompt para todas las preguntas o diferentes prompts para cada uno de los bloques del examen, el código aplicará el prompt adecuado a cada pregunta en función del valor del campo **categoría** que especifica a qué bloque del exámen pertenece la pregunta.

In [6]:
resultados = []

if len(prompts["prompts"])==1:
    multi_prompt=False
else:
    multi_prompt=True

for pregunta_id in range(len(preguntas)):
    pregunta=preguntas[pregunta_id]
    categoria=pregunta["categoria"]
    opciones_texto = "\n".join([f"{opcion['opcion']}: {opcion['texto']}" for opcion in pregunta['opciones']])
    pregunta_completa = f"{pregunta['pregunta']}\nOpciones:\n{opciones_texto}"
    prompt_list=copy.deepcopy(prompts["prompts"])

    if multi_prompt==True and categoria in prompt_list.keys():
        promptid=categoria
        prompt=prompt_list[promptid]
    else:
        promptid="comun"
        prompt=prompt_list["comun"]
       
    print("- Pregunta: {} Prompt: {}".format(pregunta_id,promptid))
    prompt=prompt_list[promptid]
    respuesta, justificacion = ejecutar_prompt(prompt, pregunta_completa)
    resultados.append({
        "pregunta": pregunta_completa,
        "prompt":promptid,
        "respuesta_simulada": respuesta,
        "respuesta_correcta": pregunta["respuesta_correcta"],
        "justificacion_simulada": justificacion
        })

- Pregunta: 0 Prompt: comun
  Done
- Pregunta: 1 Prompt: laboral
not enough values to unpack (expected 2, got 1)
not enough values to unpack (expected 2, got 1)
  Done
- Pregunta: 2 Prompt: civil_mercantil
  Done
- Pregunta: 3 Prompt: penal
  Done
- Pregunta: 4 Prompt: administrativa
  Done


Las salidas se van almacenando en la variable **resultados** que va a permitir calcular las diferentes métricas de precisión.

In [7]:
resultados

[{'pregunta': '¿Cuánto es 2 + 2?\nOpciones:\nA: 3\nB: 4\nC: 5',
  'prompt': 'comun',
  'respuesta_simulada': 'B',
  'respuesta_correcta': 'B',
  'justificacion_simulada': 'La respuesta correcta es 4, ya que al sumar 2 + 2 obtenemos como resultado 4.'},
 {'pregunta': '¿Cuánto es 5 - 3?\nOpciones:\nA: 2\nB: 3\nC: 4',
  'prompt': 'laboral',
  'respuesta_simulada': 'A',
  'respuesta_correcta': 'A',
  'justificacion_simulada': 'La resta de cinco menos tres es igual a dos.'},
 {'pregunta': '¿Cuánto  es 7 * 1?\nOpciones:\nA: 6\nB: 7\nC: 8',
  'prompt': 'civil_mercantil',
  'respuesta_simulada': 'B',
  'respuesta_correcta': 'B',
  'justificacion_simulada': 'La respuesta correcta es 7, ya que el resultado de multiplicar 7 por 1 es igual a 7.'},
 {'pregunta': '¿Cuánto  es 10 / 2?\nOpciones:\nA: 4\nB: 5\nC: 6',
  'prompt': 'penal',
  'respuesta_simulada': 'B',
  'respuesta_correcta': 'B',
  'justificacion_simulada': 'La respuesta correcta es 5, ya que 10 dividido entre 2 es igual a 5.'},
 {'pregu

Por último, esos resultados se vuelcan a un fichero de texto para que el tribunal pueda evaluar la justificación proporcionada por el LLM

In [8]:
# Guardar los resultados en un archivo de texto
with open("resultados.txt", "w") as file:
    for resultado in resultados:
        file.write(f"Pregunta: {resultado['pregunta']}\n")
        file.write(f"Respuesta correcta: {resultado['respuesta_correcta']}\n")
        file.write(f"Respuesta simulada: {resultado['respuesta_simulada']}\n")
        file.write(f"Justificación simulada: {resultado['justificacion_simulada']}\n\n")

In [9]:
# Aquí iría el cálculo de métricas basado en las respuestas simuladas.
# En una implementación real, compararíamos las respuestas simuladas con las respuestas correctas
# para calcular el número de aciertos y la puntuación.